In [1]:
%pip install numpy pillow requests imageio scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Pikachu shiny

import numpy as np
from PIL import Image
import requests
from io import BytesIO
import imageio
import os
from pathlib import Path
from sklearn.cluster import KMeans
import colorsys

def download_gif(url):
    """Baixa um GIF da URL fornecida"""
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Erro ao baixar GIF: {response.status_code}")
    return BytesIO(response.content)

def extract_frames(gif_data):
    """Extrai todos os frames de um GIF"""
    gif = Image.open(gif_data)
    frames = []
    
    try:
        for i in range(1000): # Limitamos a 1000 frames por segurança
            gif.seek(i)
            frame = gif.convert("RGBA")
            frames.append(np.array(frame))
    except EOFError:
        # Chegamos ao fim do GIF
        pass
    
    return frames, gif.info

def get_main_colors(frames, num_colors=5, mask_threshold=100):
    """Extrai as principais cores de um conjunto de frames, usando apenas pixels não transparentes"""
    all_pixels = []
    
    for frame in frames:
        # Usar apenas pixels com alpha > threshold (não transparentes)
        mask = frame[:,:,3] > mask_threshold
        pixels = frame[mask][:,:3]  # Remover canal alpha
        all_pixels.append(pixels)
    
    all_pixels = np.vstack(all_pixels)
    
    # Usar K-means para encontrar as cores principais
    kmeans = KMeans(n_clusters=num_colors, random_state=0)
    kmeans.fit(all_pixels)
    
    # Retorna as cores principais como array NumPy
    return np.array(kmeans.cluster_centers_)  # <- Esta é a linha que mudou

def create_color_mapping(normal_colors, shiny_colors):
    """Cria um mapeamento entre as cores normais e shiny"""
    # Converte para arrays NumPy se não forem
    normal_colors = np.array(normal_colors)
    shiny_colors = np.array(shiny_colors)
    
    # Ordena cores por luminosidade
    normal_colors = sorted(normal_colors, key=lambda x: 0.299*x[0] + 0.587*x[1] + 0.114*x[2])
    shiny_colors = sorted(shiny_colors, key=lambda x: 0.299*x[0] + 0.587*x[1] + 0.114*x[2])
    
    # Converte de volta para arrays para usar astype
    normal_colors = np.array(normal_colors)
    shiny_colors = np.array(shiny_colors)
    
    # Cria mapeamento
    return dict(zip(map(tuple, normal_colors.astype(int)), 
                   map(tuple, shiny_colors.astype(int))))

def is_pikachu_color(pixel, normal_colors, tolerance=30):
    """Verifica se um pixel é da cor do Pikachu"""
    # Convertemos para HSV para melhor comparação de cores
    r, g, b = pixel[:3]
    h, s, v = colorsys.rgb_to_hsv(r/255, g/255, b/255)
    
    # Amarelo do Pikachu (cor aproximada em HSV)
    # Testamos se é amarelo ou variações de cor do Pikachu
    if 0.1 < h < 0.2 and s > 0.5:  # Amarelo
        return True
    
    # Também verifica proximidade com as cores principais
    for color in normal_colors:
        dist = np.sqrt(np.sum((pixel[:3] - color)**2))
        if dist < tolerance:
            return True
            
    return False

def apply_color_mapping(frames, normal_colors, color_mapping, tolerance=30):
    """Aplica o mapeamento de cores apenas às partes amarelas do Pikachu"""
    new_frames = []
    
    for frame in frames:
        new_frame = frame.copy()
        
        # Para cada pixel
        for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                pixel = frame[i, j]
                
                # Se não for transparente e for cor do Pikachu
                if pixel[3] > 100 and is_pikachu_color(pixel, normal_colors, tolerance):
                    # Encontra a cor mais próxima no mapeamento
                    min_dist = float('inf')
                    closest_color = None
                    
                    for normal_color in color_mapping:
                        dist = np.sqrt(np.sum((pixel[:3] - normal_color)**2))
                        if dist < min_dist:
                            min_dist = dist
                            closest_color = normal_color
                    
                    # Se a distância for menor que tolerância, aplica a transformação
                    if min_dist < tolerance and closest_color is not None:
                        new_frame[i, j, :3] = color_mapping[closest_color]
        
        new_frames.append(new_frame)
    
    return new_frames

def save_gif(frames, output_path, gif_info):
    """Salva um conjunto de frames como um GIF"""
    # Converte de volta para PIL Images
    pil_frames = [Image.fromarray(frame) for frame in frames]
    
    # Salva o GIF
    pil_frames[0].save(
        output_path,
        save_all=True,
        append_images=pil_frames[1:],
        duration=gif_info.get('duration', 80),
        loop=gif_info.get('loop', 0),
        optimize=False,
        disposal=2
    )
    
    return output_path

def transform_pikachu_color(normal_url, shiny_url, cap_url, output_path="pikachu-custom.gif"):
    """Função principal que transforma o GIF"""
    # Cria a pasta de saída se não existir
    Path(os.path.dirname(output_path)).mkdir(parents=True, exist_ok=True)
    
    # Download dos GIFs
    normal_data = download_gif(normal_url)
    shiny_data = download_gif(shiny_url)
    cap_data = download_gif(cap_url)
    
    # Extrai frames
    normal_frames, _ = extract_frames(normal_data)
    shiny_frames, _ = extract_frames(shiny_data)
    cap_frames, cap_info = extract_frames(cap_data)
    
    # Obtém cores principais
    normal_colors = get_main_colors(normal_frames, num_colors=8)
    shiny_colors = get_main_colors(shiny_frames, num_colors=8)
    
    # Cria mapeamento de cores
    color_mapping = create_color_mapping(normal_colors, shiny_colors)
    
    # Aplica mapeamento aos frames do Pikachu com chapéu
    new_frames = apply_color_mapping(cap_frames, normal_colors, color_mapping)
    
    # Salva o novo GIF
    path = save_gif(new_frames, output_path, cap_info)
    print(f"GIF transformado salvo em: {path}")
    return path

# URLs dos GIFs
pikachu_normal_url = "https://play.pokemonshowdown.com/sprites/ani/pikachu.gif"
pikachu_shiny_url = "https://play.pokemonshowdown.com/sprites/ani-shiny/pikachu.gif"
pikachu_cap_url = "https://play.pokemonshowdown.com/sprites/ani-shiny/pikachu-alolacap.gif"

# Transforma o GIF
transform_pikachu_color(
    normal_url=pikachu_normal_url,
    shiny_url=pikachu_shiny_url,
    cap_url=pikachu_cap_url,
    output_path="images/0025 - Pikachu/pikachu-alola-shiny.gif"
)

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


GIF transformado salvo em: images/0025 - Pikachu/pikachu-alolacap-shiny.gif


'images/0025 - Pikachu/pikachu-alolacap-shiny.gif'